In [1]:
import os

In [2]:
import pandas as pd
import numpy as np

In [3]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [4]:
from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

In [5]:
app = Flask(__name__)

In [6]:
#################################################
# Database Setup
#################################################

In [7]:
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

In [8]:
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///Starter_Code/db/bellybutton.sqlite"
db = SQLAlchemy(app)

In [10]:
print(db)

<SQLAlchemy engine=sqlite:////Users/antonyan/Desktop/USC_Bootcamp/HOMEWORK FILES/HW 15-Plotly-Challange /Starter_Code/db/bellybutton.sqlite>


In [11]:
# reflect an existing database into a new model
Base = automap_base()

In [12]:
# reflect the tables
Base.prepare(db.engine, reflect=True)

In [18]:
# Save references to each table
Samples_Metadata = Base.classes.sample_metadata
# Samples = Base.classes.samples

AttributeError: sample_metadata

In [17]:
Samples

NameError: name 'Samples' is not defined

In [ ]:
@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")

In [ ]:
@app.route("/names")
def names():
    """Return a list of sample names."""

    # Use Pandas to perform the sql query
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Return a list of the column names (sample names)
    return jsonify(list(df.columns)[2:])

In [ ]:
@app.route("/metadata/<sample>")
def sample_metadata(sample):
    """Return the MetaData for a given sample."""
    sel = [
        Samples_Metadata.sample,
        Samples_Metadata.ETHNICITY,
        Samples_Metadata.GENDER,
        Samples_Metadata.AGE,
        Samples_Metadata.LOCATION,
        Samples_Metadata.BBTYPE,
        Samples_Metadata.WFREQ,
    ]

    results = db.session.query(*sel).filter(Samples_Metadata.sample == sample).all()
     # Create a dictionary entry for each row of metadata information
    sample_metadata = {}
    for result in results:
        sample_metadata["SAMPLE"] = result[0]
        sample_metadata["ETHNICITY"] = result[1]
        sample_metadata["GENDER"] = result[2]
        sample_metadata["AGE"] = result[3]
        sample_metadata["LOCATION"] = result[4]
        sample_metadata["BBTYPE"] = result[5]
        sample_metadata["WFREQ"] = result[6]

    print(sample_metadata)
    return jsonify(sample_metadata)

In [ ]:
@app.route("/samples/<sample>")
def samples(sample):
    """Return `otu_ids`, `otu_labels`,and `sample_values`."""
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Filter the data based on the sample number and
    # only keep rows with values above 1
    sample_data = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]
    # Format the data to send as json
    data = {
        "otu_ids": sample_data.otu_id.values.tolist(),
        "sample_values": sample_data[sample].values.tolist(),
        "otu_labels": sample_data.otu_label.tolist(),
    }
    return jsonify(data)

In [ ]:
if __name__ == "__main__":
    app.run()